In [1]:
from pymongo import MongoClient
from bson import ObjectId
import os
from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain_ollama import OllamaEmbeddings
import numpy as np
import faiss

In [2]:
load_dotenv()
URI = os.getenv("MONGODB_URI")
# Connect to MongoDB server
client = MongoClient(URI)

In [3]:
res_db = client["Res_Data"]
menu_collection = res_db["Res_menus"]

In [4]:
item = menu_collection.find_one({"_id":ObjectId('6854057e6dc04c1119f8adbf')})

In [7]:
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer('all-mpnet-base-v2', cache_folder='./models/all-mpnet-base-v2')

def get_embedding(text):
    return embed_model.encode(text)

In [8]:
def generate_embeddings_for_menu(menu_id):
    res_db = client["Res_Data"]
    menu_collection = res_db["Res_menus"]
    item = menu_collection.find_one({"_id":ObjectId(menu_id)})
    updated_sections = []
    for section in item['menu']['sections']:
        updated_dishes = []
        for dish in section['dishes']:
            # Skip if embedding already exists
            if 'embedding' in dish:
                updated_dishes.append(dish)
                continue
            
            text = f"{dish['name']}. {dish['desc']}. Category: {section}."
            embedding = get_embedding(text)
            print(f"Embedding for {dish['name']}: {embedding.tolist()}")
            dish['embedding'] = embedding.tolist()
            
            updated_dishes.append(dish)

        section['dishes'] = updated_dishes
        updated_sections.append(section)

    # Push the full nested update back
    menu_collection.update_one(
        {'_id': ObjectId(menu_id)},
        {'$set': {'menu.sections': updated_sections}}
    )

In [9]:
generate_embeddings_for_menu('6854057e6dc04c1119f8adbf')

In [ ]:
c=0
for section in item['menu']['sections']:
    print(section['name'])
    for dishes in section['dishes']:
        print(dishes['name'])
        if 'desc' in dishes:
            print(dishes['desc'])
        if 'price' in dishes:
            print(dishes['price'])
        c+=1
        print('------------------')
print(f'Total dishes: {c}')

Appetizers
Loaded Nachos
Crispy tortilla chips topped with melted cheese, jalapeños, sour cream, and salsa
7.0
------------------
Hummus & Pita Platter
Creamy hummus served with warm pita bread and fresh vegetable sticks
8.0
------------------
Sweet Potato Fries
Crispy sweet potato fries served with a tangy honey mustard dip
8.0
------------------
Buffalo Wings
Juicy chicken wings tossed in a tangy buffalo sauce, served with ranch or blue cheese dressing
8.0
------------------
Garlic Bread
Toasted baguette slices topped with garlic butter and herbs, served with marinara sauce
5.0
------------------
Chicken Tenders
Crispy breaded chicken strips served with honey mustard or barbecue sauce
7.0
------------------
Mozzarella Sticks
Golden-fried mozzarella cheese sticks served with a side of marinara sauce
5.0
------------------
Bruschetta
Grilled bread topped with diced tomatoes, garlic, basil, and a drizzle of balsamic glaze
8.0
------------------
Onion Rings
Thick-cut onions battered and 

In [ ]:
item = menu_collection.find_one({"_id":ObjectId('6854057e6dc04c1119f8adbf')})
for section in item['menu']['sections']:
    for dish in section['dishes']:
        if 'embedding' in dish:
            print(f"{dish['name']}: {dish['embedding'][:5]}...")  # Print first 5 values of the embedding
        else:
            print(f"{dish['name']}: No embedding found. Please generate embeddings first.")


In [8]:
import numpy as np

dish_records = []
menu_doc = menu_collection.find_one({"_id": ObjectId('6854057e6dc04c1119f8adbf')})

# Flatten dishes and collect embeddings
for section in menu_doc["menu"]["sections"]:
    for dish in section["dishes"]:
        if "embedding" in dish:
            dish_records.append({
                "name": dish["name"],
                "description": dish.get("desc", ""),
                "section": section["name"],
                "price": dish["price"],
                "tags": dish.get("tags", []),
                "embedding": np.array(dish["embedding"], dtype="float32")
            })

In [9]:
import faiss
dim = len(dish_records[0]["embedding"])
index = faiss.IndexFlatL2(dim)

# Add embeddings to index
embeddings = np.array([d["embedding"] for d in dish_records])
index.add(embeddings)

In [10]:
model = SentenceTransformer("all-mpnet-base-v2", cache_folder='./models/bge-small-en-v1.5')  # Or your lightweight model

# Define user query
query_text = "crispy chicken starter with sauce"

# Encode query
query_vector = model.encode(query_text).astype("float32").reshape(1, -1)

# Search FAISS index
k = 10
distances, indices = index.search(query_vector, k)

# Get top-k results
results = [dish_records[i] for i in indices[0]]
for result in results:
    print(f'Name: {result["name"]}, Section: {result["section"]}, Price: {result["price"]}, Tags: {result["tags"]} Description: {result["description"]}')

Name: Chicken Tenders, Section: Appetizers, Price: 7.0, Tags: ['meat', 'gluten'] Description: Crispy breaded chicken strips served with honey mustard or barbecue sauce
Name: Hummus & Pita Platter, Section: Appetizers, Price: 8.0, Tags: ['vegan', 'gluten'] Description: Creamy hummus served with warm pita bread and fresh vegetable sticks
Name: Chicken Caesar Salad, Section: Main Dishes, Price: 15.0, Tags: ['meat', 'dairy', 'gluten'] Description: Grilled chicken breast served on crisp romaine lettuce with Caesar dressing and croutons
Name: Buffalo Wings, Section: Appetizers, Price: 8.0, Tags: ['spicy', 'meat', 'dairy'] Description: Juicy chicken wings tossed in a tangy buffalo sauce, served with ranch or blue cheese dressing
Name: Bistro Burger, Section: Main Dishes, Price: 10.0, Tags: ['meat', 'gluten', 'dairy'] Description: Juicy beef patty, cheddar cheese, lettuce, tomato, and onion on a brioche bun. Served with fries
Name: Beef Steak Frites, Section: Special Dishes, Price: 30.0, Tags:

In [13]:
def search_menu(query_text, menu_id, k=10, threshold = 0.2):
    dish_records = []
    menu_doc = menu_collection.find_one({"_id": ObjectId(menu_id)})

    # Flatten dishes and collect embeddings
    for section in menu_doc["menu"]["sections"]:
        for dish in section["dishes"]:
            if "embedding" in dish:
                dish_records.append({
                    "name": dish["name"],
                    "description": dish.get("desc", ""),
                    "section": section["name"],
                    "price": dish["price"],
                    "tags": dish.get("tags", []),
                    "embedding": np.array(dish["embedding"], dtype="float32")
                })

    dim = len(dish_records[0]["embedding"])
    index = faiss.IndexFlatL2(dim)
    # Add embeddings to index
    embeddings = np.array([d["embedding"] for d in dish_records])
    index.add(embeddings)

    model = SentenceTransformer("all-mpnet-base-v2") 
    query_vector = model.encode(query_text).astype("float32").reshape(1, -1)
    distances, indices = index.search(query_vector, k)
    similarities = 1 - distances[0]  # Convert distance to similarity

    results = []
    for i, sim in zip(indices[0], similarities):
        if sim >= threshold:
            doc = dish_records[i]
            print(sim, ' - ', doc['name'], doc['section'], doc['price'], doc['tags'], doc['description'])
            doc.pop('embedding', None)
            results.append(doc)
    return results

In [14]:
out = search_menu(query_text = "cool drinks" , menu_id = '6854057e6dc04c1119f8adbf', k=10, threshold = -1)

-0.16926658  -  Rose Lemonade Beverages 5.0 ['cold', 'vegan', 'gluten-free'] A floral twist on classic lemonade, infused with rose essence and served over ice
-0.18399274  -  Peach Iced Tea Beverages 4.25 ['cold', 'caffeine', 'vegan', 'gluten-free'] Black tea infused with ripe peach flavor, served over ice and a slice of lemon
-0.19971251  -  Berry Lemonade Beverages 5.25 ['cold', 'vegan', 'gluten-free'] Refreshing lemonade infused with mixed berries and served over ice
-0.20209372  -  Iced Latte Beverages 4.75 ['cold', 'dairy', 'caffeine', 'gluten-free'] Chilled espresso poured over ice with cold milk for a smooth, refreshing drink
-0.24149108  -  Iced Mocha Beverages 5.25 ['cold', 'dairy', 'caffeine', 'gluten-free'] Chilled espresso blended with milk, sweet chocolate syrup, and ice
-0.29977608  -  Matcha Latte Beverages 5.0 ['cold', 'dairy', 'caffeine', 'gluten-free'] Chilled matcha green tea blended with milk and served over ice for a creamy taste
-0.30542064  -  Espresso Beverages 